<a href="https://colab.research.google.com/github/valeriojr/OtimizacaoBioinspirada/blob/main/%5BBioinspiradas%5D_GA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Alterações realizadas no código

*   Separar imports para adicionar código
*   **1. Mudar a representação de binário para "Gray coding" (slides 30 e 31)**
    *       Adiciona a função `gray_repr` que converte uma *string* no formato binário para o equivalente na codificação de Gray
    *       Função `convertToGray` para converter uma população para a codificação de Gray
    *       Adiciona a chamada da função `convertToGray` para converter a população gerada na função `initialPopulation`
*    **2. Trocar a seleção dos pais para ser feito por meio de “Tournament selection” (slide 37)**
    *       Adiciona a função `tournament_selection`
    *       Altera o método de seleção de `parents_selection` para `tournment_selection` na linha 12 de https://colab.research.google.com/drive/1wS29O64GJ--rCtCOQgnVAyP6pnViV41b#scrollTo=zel-fczmtYrB&line=9&uniqifier=1 
*    **3. Trocar o crossover de dois pontos para o "Uniform crossover"**
    *       Adiciona a função `uniform_crossover`
    *       Altera o método de cruzamento de `crossover` para `uniform_crossover` na linha 14 de https://colab.research.google.com/drive/1wS29O64GJ--rCtCOQgnVAyP6pnViV41b#scrollTo=zel-fczmtYrB&line=14&uniqifier=1
*    **4. Criar script que executa o GA com população de 10, 15 e 20 indivíduos.**
    *         

In [42]:
import random
import numpy as np
import math as math
import itertools

In [43]:
def gray_repr(var, width):
  bits = range(1, len(var))
  msb = var.find('1')
  gray = [var[0], *('1' if var[i] != var[i - 1] else '0' for i in bits)]
  gray[msb] = var[msb]
  return ''.join(gray)

def convertToGray(population, width):
  args = zip(population, itertools.cycle([width]))
  return list(map(lambda args: gray_repr(args[0], args[1]), args))
  
convertToGray(['0000', '0101', '1000', '0010', '0110'], 8)

['0000', '0111', '1100', '0011', '0101']

In [44]:
def converToBin(population,width):
  populationBin = []
  for individual in population:
    varBin = ''
    for variable in individual:
      varBin += np.binary_repr(variable, width) #vetor de caracter '0' ou '1'
    populationBin.append(varBin)
  return populationBin

def initialPopulation(populationSize, upperBound, lowerBound, bitWidth):
  population = []
  numberVar = len(upperBound)
  for i in range(populationSize):
    individual = []
    for v in range(numberVar):
      individual.append(random.randint(lowerBound[v], upperBound[v]))
    population.append(individual)
  return convertToGray(converToBin(population,bitWidth), bitWidth)

In [45]:
def exampleProblem(individual):
  value = int(individual, 2)
  return math.sin(math.pi*value/256)

def avalFitness(population):
    populationFitness = []
    for individual in population:        
      populationFitness.append(exampleProblem(individual))
    return populationFitness

In [54]:
# REPRODUCAO
def normalizeFitness(populationFitness):
  fitnessSum = sum(populationFitness)
  norm_fitness = []
  for fitness in populationFitness:
    norm_fitness.append(fitness/fitnessSum)
  return norm_fitness

def cumulativeFitness(normalizedFitness):
    cumulativeFitness = []
    cumulative = 0
    for fitness in normalizedFitness:
        cumulative = fitness + cumulative
        cumulativeFitness.append(cumulative)
    return cumulativeFitness

def parents_selection(population, populationFitness):
  normalizedFitness = normalizeFitness(populationFitness)
  cFitness = cumulativeFitness(normalizedFitness)
  parents = []

  for i in range(len(population)):
    rand = random.random()
    for j in range(len(cFitness)):
      if cFitness[j] >= rand:
        parents.append(population[j])
        break

  return parents

def tournament_selection(population, populationFitness):
  # Cria os indices de cada individuo: 0, 1, ..., (#population - 1) e 
  # embaralha esses indices (serão divididos em 2 equipes)
  pairs = np.arange(len(population))
  np.random.shuffle(pairs)
  # Quantidade de membros em cada equipe (caso o numero de participantes seja
  # impar o ultimo será descartado)
  split = len(pairs) // 2
  # Filtra os individuos no mata mata: pega um membro de cada equipe e o que
  # tiver maior fitness vence e é selecionado
  return [population[individualA] if populationFitness[individualA] > populationFitness[individualB] else population[individualB]
    for individualA, individualB in zip(pairs[:split], pairs[split:])]



In [47]:
#CRUZAMENTO
def selectPairs(parents):
  pairs = []
  for i in range(int(len(parents)/2)):
    pair = []
    pair.append(parents[random.randint(0,len(parents)-1)])
    pair.append(parents[random.randint(0,len(parents)-1)])
    pairs.append(pair)
  return pairs

def crossover(parents, crossoverProbability):
  pairs = selectPairs(parents)
  newPopulation = []
  for pair in pairs: 
    breakpoint1 = random.randint(1, len(pair[0])-1)
    breakpoint2 = random.randint(1, len(pair[0])-1)

    while breakpoint1 == breakpoint2:
      breakpoint2 = random.randint(1, len(pair[0]))
    
    if (random.random() > crossoverProbability): 
      newPopulation.append(pair[0])
      newPopulation.append(pair[1])
      continue

    if breakpoint1 < breakpoint2: 
      offspring1 = pair[0][0:breakpoint1] + pair[1][breakpoint1:breakpoint2] + pair[0][breakpoint2:]
      offspring2 = pair[1][0:breakpoint1] + pair[0][breakpoint1:breakpoint2] + pair[1][breakpoint2:]
    else:
      offspring1 = pair[1][0:breakpoint2] + pair[0][breakpoint2:breakpoint1] + pair[1][breakpoint1:]
      offspring2 = pair[0][0:breakpoint2] + pair[1][breakpoint2:breakpoint1] + pair[0][breakpoint1:]
    
    newPopulation.append(offspring1)
    newPopulation.append(offspring2)
      
  return newPopulation

def uniform_crossover(parents, crossoverProbability):
  def do_crossover(individualA, individualB):
      newIndividualA, newIndividualB = [], []
      for geneA, geneB in zip(individualA, individualB):
        if random.random() > 0.5:
            geneA, geneB = geneB, geneA
        
        newIndividualA.append(geneA)
        newIndividualB.append(geneB)
      
      return ''.join(newIndividualA), ''.join(newIndividualB)

  pairs = selectPairs(parents)
  keep_original = [random.range() > crossoverProbability for _ in range(len(pairs))]
  return [pair if keep_original[i] else do_crossover(*pair) for i, pair in enumerate(pairs)]

In [48]:
#MUTAÇÃO
def mutation(population,mutationProbability):
  newPop = []
  for individual in population:
    listInd = list(individual)
    for i in range(len(listInd)):    
      if random.random() < mutationProbability:
        if listInd[i] == '0':
          listInd[i] = '1'
        else:
          listInd[i] = '0'
    newPop.append(''.join(listInd))
  return newPop

In [50]:
def GA(upperBound,lowerBound,bitWidth=8,max_iterations=20,crossoverProb=0.8,mutationProb=0.01,populationSize=50):
  #gerar populacao inicial
  population = initialPopulation(populationSize,upperBound,lowerBound,bitWidth)
  current_iteration = 0
  while(current_iteration < max_iterations):
    #avaliar o fitness
    populationFitness = avalFitness(population)

    print('#', current_iteration, ' Best fitness = ', max(populationFitness), '  ', int(population[populationFitness.index(max(populationFitness))], base=2))

  	#selecao dos pais
    parents = tournament_selection(population,populationFitness)
  	#cruzamento
    population = uniform_crossover(parents, crossoverProb)
  	#mutação
    population = mutation(population, mutationProb)
  	
    current_iteration += 1
  return

In [ ]:
lowerBound = [0]
upperBound = [256]
for i, populationSize in enumerate([10, 15, 20]):
    print(f'Run #{i}\nPopulation Size: {populationSize}\n')
    GA(upperBound,lowerBound,populationSize=8)